In [ ]:
%matplotlib inline

import json
import datetime
import os
import re
import urllib

import numpy as np
import matplotlib.pyplot as plt

import backend
import frontend

for module in (frontend, backend):
    module = reload(module)

In [ ]:
backend_times, _ = backend.get_times('20161002', '140000-203000')
frontend_times = frontend.get_times('20161002', '140000-203000')

In [ ]:
backend_times_ok = [t for t in backend_times if t[3]['metadata']['status'][0] == '200']
backend_times_bad = [t for t in backend_times if t[3]['metadata']['status'][0] != '200']

text = '{} reqs'.format(len(frontend_times))
text += '\n{} slow reqs'.format(len(backend_times_ok))
text += '\n{} bad reqs'.format(len(backend_times_bad))
text += '\n{:1.2f} med resp time'.format(backend_times_ok[len(backend_times_ok) / 2][1])
text += '\n{:1.2f} max resp time'.format(max([t[1] for t in backend_times_ok]))

In [ ]:
ok_pct = 100 * float(len(backend_times_ok)) / len(frontend_times)
bad_pct = 100 * float(len(backend_times_bad)) / len(frontend_times)
labels = 'T < 0.5s', 'T >= 0.5s (%1.2f%%)' % ok_pct, 'Erro (%1.2f%%)' % bad_pct
pie_slices = [len(frontend_times) - len(backend_times), len(backend_times_ok), len(backend_times_bad)]
colors = ['yellowgreen', 'yellow', 'coral']

fig, ax = plt.subplots(1, 1, figsize=(16, 9))
patches, texts1, texts2 = plt.pie(pie_slices, colors=colors, autopct='', shadow=True, startangle=90)
plt.title(u'\nBackend – Requests 16:00 ~ 20:30\n')
plt.legend(patches, labels)
axis = plt.axis('equal')

props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
title = ax.text(-1.5, 0.95, text, fontsize=14, verticalalignment='top', bbox=props)

In [ ]:
days = ['20160926', '20160927', '20160928', '20160929', '20160930', '20161001', '20161002']
history_backend = []
history_frontend = []
for day in days:
    times = frontend.get_times(day)
    history_frontend.append((day, times))
    times, canceled = backend.get_times(day)
    history_backend.append((day, times, canceled))

In [ ]:
medians = []
errors = []
maxes = []
times_label = []
for i, (day, times, canceled) in enumerate(history_backend):
    total_reqs = len(history_frontend[i][1])
    slow_reqs = [t[1] for t in times if t[3]['metadata']['status'][0] == '200']
    bad_reqs = [t[1] for t in times if t[3]['metadata']['status'][0] != '200']
    len_slow_reqs = len(slow_reqs)
    len_bad_reqs = len(bad_reqs)
    median_ = np.median(slow_reqs)
    max_ = max(slow_reqs)
    medians.append(median_)
    errors.append(len_bad_reqs)
    maxes.append(max_)
    times_label.append((
        total_reqs,
        len_slow_reqs,
        len_bad_reqs,
        100 * float(len_slow_reqs) / total_reqs,
        100 * float(len_bad_reqs) / total_reqs)
    )

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 10))
index = np.arange(len(history_backend))
bar_width = 0.3

rects1 = plt.bar(index, medians, bar_width,
                 alpha=0.33,
                 color='b',
                 label='mdiana')

rects2 = plt.bar(index+bar_width, maxes, bar_width,
                 alpha=0.33,
                 color='r',
                 label='max')

ax.set_xticklabels([
    '{0}{1}\n{0}{2} [{3} / {4}]\n{0}{5:1.2f}% / {6:1.2f}%'.format(' '*38, t[0], *times_label[i])
    for i, t in enumerate(history_backend)
])

ax.legend((rects1[0], rects2[0]), ('median', 'max'))
text = plt.title(u'\nBackend – Requests 16:00 ~ 20:30 t >= 0.5s\n')

In [ ]:
date_time_str = backend_times[0][3]['date'] + backend_times[0][3]['time'][:-1]
good_times = []
bad_times = []
good_reqs = []
bad_reqs = []
good_times_medians = []
bad_times_medians = []
for feed, time, when, item, date_time in backend_times:
    if item['metadata']['status'][0] == '200':
        good_times.append(time)
    else:
        bad_times.append(time)
    if date_time_str != item['date'] + item['time'][:-1]:
        good_reqs.append((date_time, len(good_times)))
        bad_reqs.append((date_time, len(bad_times)))
        good_times_medians.append((date_time, np.median(good_times)))
        bad_times_medians.append((date_time, np.median(bad_times)))
        good_times = []
        bad_times = []
        date_time_str = item['date'] + item['time'][:-1]
else:
    if good_times or bad_times:
        good_reqs.append((date_time, len(good_times)))
        bad_reqs.append((date_time, len(bad_times)))
        good_times_medians.append((date_time, np.median(good_times)))
        bad_times_medians.append((date_time, np.median(bad_times)))

date_time_str = frontend_times[0][3]['date'] + frontend_times[0][3]['time'][:-2]
total_reqs = []
reqs = 0
for feed, time, when, item, date_time in frontend_times:
    reqs += 1
    if date_time_str != item['date'] + item['time'][:-2]:
        total_reqs.append((date_time, reqs))
        date_time_str = item['date'] + item['time'][:-2]
        reqs = 0
else:
    if reqs:
        total_reqs.append((date_time, reqs))

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(25, 15))
ax1.plot([m[0] for m in good_times_medians], [m[1] for m in good_times_medians], 'g')
ax1.plot([m[0] for m in bad_times_medians], [m[1] for m in bad_times_medians], 'r')
ax2.plot([r[0] for r in good_reqs], [r[1] for r in good_reqs], 'g')
ax2.plot([r[0] for r in bad_reqs], [r[1] for r in bad_reqs], 'r')
ax3.plot([t[0] for t in total_reqs], [t[1] for t in total_reqs])
ax1.set_ylabel('response time')
ax2.set_ylabel('num of requests')
_ = ax3.set_ylabel('num of requests')